In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
import copy

# Bike Data

In [2]:
bikes = pd.read_csv('Eco-Totem_Capital_City_Trail_Bike_Counts.csv')
bikes["Date"] = pd.to_datetime(bikes["Count_Date"])
bikes["month"] = bikes.Date.dt.month
bikes["year"] = bikes.Date.dt.year
bikes["day"] = bikes.Date.dt.day
bikes["day_of_week"] = bikes.Date.dt.dayofweek

In [3]:
def map_season(x):
    if x < 3 or x > 11:
        return "winter"
    elif x >= 3 and x <= 4:
        return "spring"
    elif x >= 5 and x <= 8:
        return "summer"
    else:
        return "fall"
    
def season_code(x):
    if x == 'winter':
        return 4
    elif x == 'spring':
        return 1
    elif x == 'summer':
        return 2
    else:
        return 3
    
bikes["season"] = bikes["month"].apply(map_season)
bikes["season code"] = bikes["season"].apply(season_code)

In [4]:
daily_bikes = bikes.groupby(["year","month","day"]).mean()["Count"].reset_index()
daily_bikes["test"] = None
for idx, row in daily_bikes.iterrows():
    daily_bikes['test'].loc[idx] = str(row['year']) + '/'+ str(row['month']) + '/' + str(row['day'])
daily_bikes["Date"] = pd.to_datetime(daily_bikes['test'])

C:\Users\cs301\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
daily_bikes["season"] = daily_bikes["month"].apply(map_season)
daily_bikes["season code"] = daily_bikes["season"].apply(season_code)
daily_bikes.head()

,year,month,day,Count,test,Date,season,season code
0,2015,6,23,203.307692,2015/6/23,2015-06-23,summer,2
1,2015,6,24,129.416667,2015/6/24,2015-06-24,summer,2
2,2015,6,25,91.375000,2015/6/25,2015-06-25,summer,2
3,2015,6,26,106.541667,2015/6/26,2015-06-26,summer,2
4,2015,6,27,164.666667,2015/6/27,2015-06-27,summer,2


# Weather Data

In [6]:
weather = pd.read_csv('Weather Data.csv')
weather = weather[weather['STATION'] == 'USW00014837']
weather = weather.groupby("DATE").mean()[['TMAX', 'TMIN', 'PRCP', 'SNOW']]
weather

,TMAX,TMIN,PRCP,SNOW
DATE,,,,
1/1/2015,31.0,14.0,0.00,0.0
1/1/2016,28.0,17.0,0.00,0.0
1/1/2017,36.0,17.0,0.00,0.0
1/1/2018,2.0,-13.0,0.00,0.0
1/1/2019,30.0,19.0,0.00,0.0
1/1/2020,40.0,16.0,0.00,0.0
1/10/2015,23.0,-3.0,0.00,0.0
1/10/2016,14.0,-4.0,0.00,0.0
1/10/2017,43.0,16.0,0.58,0.0


In [7]:
weather["Date"] = pd.to_datetime(weather.index)
weather["Month"] = weather.Date.dt.month
weather["Year"] = weather.Date.dt.year
weather.sort_values(by = "Date")
weather.head()

,TMAX,TMIN,PRCP,SNOW,Date,Month,Year
DATE,,,,,,,
1/1/2015,31.0,14.0,0.0,0.0,2015-01-01,1,2015
1/1/2016,28.0,17.0,0.0,0.0,2016-01-01,1,2016
1/1/2017,36.0,17.0,0.0,0.0,2017-01-01,1,2017
1/1/2018,2.0,-13.0,0.0,0.0,2018-01-01,1,2018
1/1/2019,30.0,19.0,0.0,0.0,2019-01-01,1,2019


In [8]:
other_weather = pd.read_csv("weather.csv")

In [9]:
other_weather["Date"] = pd.to_datetime(other_weather.date)
other_weather["Month"] = other_weather.Date.dt.month
other_weather["Year"] = other_weather.Date.dt.year
other_weather.sort_values(by = "Date")
other_weather

,time,temp,dew,humidity,wind,windspeed,gust,pressure,precip,cond,date,hour,Date,Month,Year
0,12:53 AM,59,54,83,N,3,0,29.00,0.0,Mostly Cloudy,2015-7-1,1,2015-07-01,7,2015
1,1:18 AM,59,54,83,NNE,5,0,28.99,0.0,Mostly Cloudy,2015-7-1,1,2015-07-01,7,2015
2,1:37 AM,59,55,87,CALM,0,0,29.00,0.0,Cloudy,2015-7-1,2,2015-07-01,7,2015
3,1:53 AM,60,55,83,CALM,0,0,29.00,0.0,Cloudy,2015-7-1,2,2015-07-01,7,2015
4,2:53 AM,61,57,87,CALM,0,0,29.01,0.0,Cloudy,2015-7-1,3,2015-07-01,7,2015
5,3:06 AM,60,53,78,NE,14,17,29.02,0.0,Cloudy,2015-7-1,3,2015-07-01,7,2015
6,3:53 AM,59,47,64,NE,8,0,29.02,0.0,Cloudy,2015-7-1,4,2015-07-01,7,2015
7,4:53 AM,56,48,75,N,7,0,29.03,0.0,Cloudy,2015-7-1,5,2015-07-01,7,2015
8,5:53 AM,56,48,75,NNE,7,0,29.04,0.0,Cloudy,2015-7-1,6,2015-07-01,7,2015
9,6:53 AM,56,46,69,NNE,8,0,29.06,0.0,Mostly Cloudy,2015-7-1,7,2015-07-01,7,2015


In [10]:
other_weather = other_weather[(other_weather['hour']) >=6]
other_weather = other_weather[(other_weather['hour'] <=22)]
other_weather = other_weather.groupby("Date").mean()[['windspeed', 'humidity']]
other_weather

,windspeed,humidity
Date,,
2015-07-01,5.944444,58.944444
2015-07-02,6.666667,59.611111
2015-07-03,2.588235,58.529412
2015-07-04,2.555556,61.000000
2015-07-05,9.058824,61.294118
2015-07-06,7.971429,83.542857
2015-07-07,9.612903,72.064516
2015-07-08,4.529412,63.470588
2015-07-09,1.823529,60.705882


# COVID-19 Data

In [11]:
cases = pd.read_csv('Cumulative_cases_data (1).csv')
cases["Date"] = pd.to_datetime(cases["Day of Epi Dt"])
cases["month"] = cases.Date.dt.month
cases["year"] = cases.Date.dt.year
cases["day"] = cases.Date.dt.day
cases["Confirmed Cases"] = cases['Confirmed Cases By Epi Dt']
cases = cases.drop(columns =['Day of Epi Dt','County Parameter Cum Curve', 'Running Sum of Confirmed Cases By Epi Dt', 'Confirmed Cases By Epi Dt'])
cases.head()

,Date,month,year,day,Confirmed Cases
0,2020-01-22,1,2020,22,0
1,2020-01-23,1,2020,23,0
2,2020-01-24,1,2020,24,0
3,2020-01-25,1,2020,25,0
4,2020-01-26,1,2020,26,0


In [12]:
cases['Confirmed Cases']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        1
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
      ... 
218     86
219     66
220     66
221     82
222    175
223    173
224    207
225    200
226    201
227    137
228    136
229    148
230    322
231    255
232    177
233    162
234    106
235     76
236    200
237    164
238    163
239    140
240    106
241     67
242     60
243    122
244    100
245     58
246     16
247     13
Name: Confirmed Cases, Length: 248, dtype: int64

# Merge Data

In [13]:
merged_df = daily_bikes.merge(weather, on = 'Date')
merged_df = merged_df.merge(other_weather, on = 'Date')
merged_df

,year,month,day,Count,test,Date,season,season code,TMAX,TMIN,PRCP,SNOW,Month,Year,windspeed,humidity
0,2015,7,1,162.500000,2015/7/1,2015-07-01,summer,2,71.0,53.0,0.00,0.0,7,2015,5.944444,58.944444
1,2015,7,2,134.458333,2015/7/2,2015-07-02,summer,2,72.0,46.0,0.00,0.0,7,2015,6.666667,59.611111
2,2015,7,3,138.041667,2015/7/3,2015-07-03,summer,2,77.0,49.0,0.00,0.0,7,2015,2.588235,58.529412
3,2015,7,4,142.791667,2015/7/4,2015-07-04,summer,2,82.0,56.0,0.00,0.0,7,2015,2.555556,61.000000
4,2015,7,5,136.625000,2015/7/5,2015-07-05,summer,2,83.0,61.0,0.00,0.0,7,2015,9.058824,61.294118
5,2015,7,6,64.458333,2015/7/6,2015-07-06,summer,2,80.0,62.0,0.77,0.0,7,2015,7.971429,83.542857
6,2015,7,7,123.916667,2015/7/7,2015-07-07,summer,2,71.0,56.0,0.00,0.0,7,2015,9.612903,72.064516
7,2015,7,8,136.041667,2015/7/8,2015-07-08,summer,2,70.0,51.0,0.00,0.0,7,2015,4.529412,63.470588
8,2015,7,9,150.458333,2015/7/9,2015-07-09,summer,2,79.0,53.0,0.00,0.0,7,2015,1.823529,60.705882
9,2015,7,10,151.583333,2015/7/10,2015-07-10,summer,2,82.0,56.0,0.00,0.0,7,2015,3.294118,58.117647


In [14]:
test = copy.copy(merged_df)
test1 = test[test['Date'] < pd.to_datetime('2020-01-22')]
test2 = test[test['Date'] >= pd.to_datetime('2020-01-22')]
test2

,year,month,day,Count,test,Date,season,season code,TMAX,TMIN,PRCP,SNOW,Month,Year,windspeed,humidity
1639,2020,1,22,18.666667,2020/1/22,2020-01-22,winter,4,31.0,21.0,0.02,0.2,1,2020,12.750000,74.200000
1640,2020,1,23,16.375000,2020/1/23,2020-01-23,winter,4,32.0,28.0,0.08,1.3,1,2020,6.435897,89.307692
1641,2020,1,24,14.708333,2020/1/24,2020-01-24,winter,4,34.0,32.0,0.25,2.1,1,2020,8.821429,93.964286
1642,2020,1,25,5.375000,2020/1/25,2020-01-25,winter,4,36.0,32.0,0.20,3.5,1,2020,6.500000,91.000000
1643,2020,1,26,5.833333,2020/1/26,2020-01-26,winter,4,33.0,27.0,0.00,0.0,1,2020,5.545455,86.090909
1644,2020,1,27,19.916667,2020/1/27,2020-01-27,winter,4,33.0,26.0,0.00,0.1,1,2020,4.037037,85.148148
1645,2020,1,28,20.916667,2020/1/28,2020-01-28,winter,4,31.0,24.0,0.00,0.0,1,2020,5.105263,77.526316
1646,2020,1,29,20.291667,2020/1/29,2020-01-29,winter,4,26.0,23.0,0.00,0.0,1,2020,2.157895,77.157895
1647,2020,1,30,20.291667,2020/1/30,2020-01-30,winter,4,30.0,24.0,0.00,0.0,1,2020,2.850000,75.550000
1648,2020,1,31,21.041667,2020/1/31,2020-01-31,winter,4,33.0,27.0,0.01,0.1,1,2020,6.500000,87.615385


In [15]:
test3 = test2.merge(cases, on = 'Date')
test3

,year_x,month_x,day_x,Count,test,Date,season,season code,TMAX,TMIN,PRCP,SNOW,Month,Year,windspeed,humidity,month_y,year_y,day_y,Confirmed Cases
0,2020,1,22,18.666667,2020/1/22,2020-01-22,winter,4,31.0,21.0,0.02,0.2,1,2020,12.750000,74.200000,1,2020,22,0
1,2020,1,23,16.375000,2020/1/23,2020-01-23,winter,4,32.0,28.0,0.08,1.3,1,2020,6.435897,89.307692,1,2020,23,0
2,2020,1,24,14.708333,2020/1/24,2020-01-24,winter,4,34.0,32.0,0.25,2.1,1,2020,8.821429,93.964286,1,2020,24,0
3,2020,1,25,5.375000,2020/1/25,2020-01-25,winter,4,36.0,32.0,0.20,3.5,1,2020,6.500000,91.000000,1,2020,25,0
4,2020,1,26,5.833333,2020/1/26,2020-01-26,winter,4,33.0,27.0,0.00,0.0,1,2020,5.545455,86.090909,1,2020,26,0
5,2020,1,27,19.916667,2020/1/27,2020-01-27,winter,4,33.0,26.0,0.00,0.1,1,2020,4.037037,85.148148,1,2020,27,0
6,2020,1,28,20.916667,2020/1/28,2020-01-28,winter,4,31.0,24.0,0.00,0.0,1,2020,5.105263,77.526316,1,2020,28,0
7,2020,1,29,20.291667,2020/1/29,2020-01-29,winter,4,26.0,23.0,0.00,0.0,1,2020,2.157895,77.157895,1,2020,29,1
8,2020,1,30,20.291667,2020/1/30,2020-01-30,winter,4,30.0,24.0,0.00,0.0,1,2020,2.850000,75.550000,1,2020,30,0
9,2020,1,31,21.041667,2020/1/31,2020-01-31,winter,4,33.0,27.0,0.01,0.1,1,2020,6.500000,87.615385,1,2020,31,0


In [16]:
test1['Confirmed Cases'] = pd.Series([0 for x in range(len(test1.index))])
test1

C:\Users\cs301\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,year,month,day,Count,test,Date,season,season code,TMAX,TMIN,PRCP,SNOW,Month,Year,windspeed,humidity,Confirmed Cases
0,2015,7,1,162.500000,2015/7/1,2015-07-01,summer,2,71.0,53.0,0.00,0.0,7,2015,5.944444,58.944444,0
1,2015,7,2,134.458333,2015/7/2,2015-07-02,summer,2,72.0,46.0,0.00,0.0,7,2015,6.666667,59.611111,0
2,2015,7,3,138.041667,2015/7/3,2015-07-03,summer,2,77.0,49.0,0.00,0.0,7,2015,2.588235,58.529412,0
3,2015,7,4,142.791667,2015/7/4,2015-07-04,summer,2,82.0,56.0,0.00,0.0,7,2015,2.555556,61.000000,0
4,2015,7,5,136.625000,2015/7/5,2015-07-05,summer,2,83.0,61.0,0.00,0.0,7,2015,9.058824,61.294118,0
5,2015,7,6,64.458333,2015/7/6,2015-07-06,summer,2,80.0,62.0,0.77,0.0,7,2015,7.971429,83.542857,0
6,2015,7,7,123.916667,2015/7/7,2015-07-07,summer,2,71.0,56.0,0.00,0.0,7,2015,9.612903,72.064516,0
7,2015,7,8,136.041667,2015/7/8,2015-07-08,summer,2,70.0,51.0,0.00,0.0,7,2015,4.529412,63.470588,0
8,2015,7,9,150.458333,2015/7/9,2015-07-09,summer,2,79.0,53.0,0.00,0.0,7,2015,1.823529,60.705882,0
9,2015,7,10,151.583333,2015/7/10,2015-07-10,summer,2,82.0,56.0,0.00,0.0,7,2015,3.294118,58.117647,0


In [17]:
final_df = pd.concat([test1, test3], ignore_index=True)
final_df = final_df.drop(columns = ['day_x', 'day_y','month', 'month_x', 'month_y','day', 'test', 'year_x', 'year_y', 'year', 'Month'])
final_df

C:\Users\cs301\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Confirmed Cases,Count,Date,PRCP,SNOW,TMAX,TMIN,Year,humidity,season,season code,windspeed
0,0,162.500000,2015-07-01,0.00,0.0,71.0,53.0,2015,58.944444,summer,2,5.944444
1,0,134.458333,2015-07-02,0.00,0.0,72.0,46.0,2015,59.611111,summer,2,6.666667
2,0,138.041667,2015-07-03,0.00,0.0,77.0,49.0,2015,58.529412,summer,2,2.588235
3,0,142.791667,2015-07-04,0.00,0.0,82.0,56.0,2015,61.000000,summer,2,2.555556
4,0,136.625000,2015-07-05,0.00,0.0,83.0,61.0,2015,61.294118,summer,2,9.058824
5,0,64.458333,2015-07-06,0.77,0.0,80.0,62.0,2015,83.542857,summer,2,7.971429
6,0,123.916667,2015-07-07,0.00,0.0,71.0,56.0,2015,72.064516,summer,2,9.612903
7,0,136.041667,2015-07-08,0.00,0.0,70.0,51.0,2015,63.470588,summer,2,4.529412
8,0,150.458333,2015-07-09,0.00,0.0,79.0,53.0,2015,60.705882,summer,2,1.823529
9,0,151.583333,2015-07-10,0.00,0.0,82.0,56.0,2015,58.117647,summer,2,3.294118


In [18]:
final_df.to_csv("Modeling Data.csv")